In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import os
from os.path import exists
from os import remove
import sklearn.linear_model as lm
import json
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
import sklearn.model_selection as ms
from sklearn.model_selection import cross_validate
from sklearn.feature_selection import f_regression

import sys
sys.path.insert(0, "../util/")
import util as util

/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (36,40,81,87) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


# Add Dummies and Train-Test-Split

In [2]:
housingTrainWithDummiesExists = exists('../data/housingTrainWithDummies.csv')
housingTestWithDummiesExists = exists('../data/housingTestWithDummies.csv')
housingTrainWithoutDummiesExists = exists('../data/housingTrainWithoutDummies.csv')
housingTestWithoutDummiesExists = exists('../data/housingTestWithoutDummies.csv')
relatedDummiesDictionaryExists = exists('../data/relatedDummiesDictionary.json')

if (housingTrainWithDummiesExists and housingTestWithDummiesExists and
   housingTrainWithoutDummiesExists and housingTestWithoutDummiesExists and
   relatedDummiesDictionaryExists):
    housingTrainWithDummies = pd.read_csv('../data/housingTrainWithDummies.csv')
    housingTestWithDummies = pd.read_csv('../data/housingTestWithDummies.csv')
    housingTrainWithoutDummies = pd.read_csv('../data/housingTrainWithoutDummies.csv')
    housingTestWithoutDummies = pd.read_csv('../data/housingTestWithoutDummies.csv')
    with open('../data/relatedDummiesDictionary.json') as d:
        relatedDummiesDictionary = json.load(d)
    for df in [housingTrainWithDummies,housingTestWithDummies,housingTrainWithoutDummies,housingTestWithoutDummies]:
        df["MSSubClass"] = df["MSSubClass"].apply(str)
    
else:
    if (housingTrainWithDummiesExists):
        remove('../data/housingTrainWithDummies.csv')
    if (housingTestWithDummiesExists):
        remove('../data/housingTestWithDummies.csv')
    if (housingTrainWithoutDummiesExists):
        remove('../data/housingTrainWithoutDummies.csv')
    if (housingTestWithoutDummiesExists):
        remove('../data/housingTestWithoutDummies.csv')
    if (relatedDummiesDictionaryExists):
        remove('../data/relatedDummiesDictionary.json')
    
    amesHousing = pd.read_csv('../data/Ames_Housing_Price_Data.csv')   
    
    amesHousing['MSSubClass'] = amesHousing['MSSubClass'].apply(str)
    
    amesHousingCategoricalData = amesHousing.select_dtypes(include=['O'])
    for col in amesHousingCategoricalData.columns:
        amesHousing[col]=amesHousing[col].fillna('nan')
    relatedDummiesDictionary = {}
    for col in amesHousingCategoricalData.columns:
        dummyData = pd.get_dummies(amesHousing[col],prefix=col,drop_first=True)
        for dummyCol in dummyData.columns:
            relatedDummiesDictionary[str(dummyCol)] = list(dummyData.columns)
        amesHousing = pd.concat([amesHousing,dummyData],axis=1)
    with open('../data/relatedDummiesDictionary.json', 'w') as fp:
        json.dump(relatedDummiesDictionary, fp)
    
    salesPrice = amesHousing['SalePrice']
    X = amesHousing.drop(['SalePrice'], axis=1)
    
    housingTrainWithDummies, housingTestWithDummies, salesPriceTrain, salesPriceTest = train_test_split(X, salesPrice, test_size=0.2)
    
    housingTrainWithDummies['SalePrice'] = salesPriceTrain
    housingTestWithDummies['SalePrice'] = salesPriceTest
    
    dummyColumns = list(relatedDummiesDictionary.keys())
    housingTrainWithoutDummies = housingTrainWithDummies.drop(dummyColumns, axis=1)
    housingTestWithoutDummies = housingTestWithDummies.drop(dummyColumns, axis=1)
    
    housingTrainWithDummies.to_csv('../data/housingTrainWithDummies.csv')
    housingTestWithDummies.to_csv('../data/housingTestWithDummies.csv')
    housingTrainWithoutDummies.to_csv('../data/housingTrainWithoutDummies.csv')
    housingTestWithoutDummies.to_csv('../data/housingTestWithoutDummies.csv')

/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (75) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
if (("ISUDistance" not in housingTrainWithoutDummies.columns) or 
    ("ISUDistance" not in housingTrainWithDummies.columns)):
    housingTrainWithoutDummies = util.returnDFWithISUDistance(housingTrainWithoutDummies,True)
    housingTrainWithDummies["ISUDistance"] = housingTrainWithoutDummies["ISUDistance"]
    housingTrainWithDummies.to_csv('../data/housingTrainWithDummies.csv')
    housingTrainWithoutDummies.to_csv('../data/housingTrainWithoutDummies.csv')

# Training

## Linear Regression

In [4]:
housingTrainNumerical = housingTrainWithDummies.select_dtypes(include=['uint8','int64','float64'])

In [5]:
if (exists('../data/trainNanReplacementValuesDictionary.json')):
    with open('../data/trainNanReplacementValuesDictionary.json') as d:
        trainNanReplacementValuesDictionary = json.load(d)
else:
    trainNanReplacementValuesDictionary = {}
    for col in housingTrainNumerical:
        if col in ["YearBuilt","YearRemodAdd","GarageYrBlt","YrSold","ISUDistance"]:
            trainNanReplacementValuesDictionary[str(col)] = housingTrainNumerical[col].mean()
        elif col in ["OverallQual","OverallCond","MoSold"]:
            trainNanReplacementValuesDictionary[str(col)] = round(np.mean(housingTrainNumerical[col].mode().values))
        else:
            trainNanReplacementValuesDictionary[str(col)] = 0
    with open('../data/trainNanReplacementValuesDictionary.json', 'w') as fp:
        json.dump(trainNanReplacementValuesDictionary, fp)
    

In [6]:
housingTrainNumerical = util.replaceNansWithTrainingDataValues(housingTrainNumerical)

../util/util.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna(trainNanReplacementValuesDictionary[str(col)])


In [7]:
salesPrice = housingTrainNumerical["SalePrice"]
X = housingTrainNumerical.drop(['SalePrice'], axis=1)

In [8]:
initialAlpha = [1]+list(np.linspace(50,1000,20))

In [9]:
grid_para_elsatic_net_initial = {'alpha': initialAlpha, 'l1_ratio': np.linspace(0,1,11)}

In [10]:
elasticNet = ElasticNet()
if False:
    grid_search_elastic_net = ms.GridSearchCV(elasticNet, grid_para_elsatic_net_initial, scoring='accuracy', cv=5, n_jobs=-1, return_train_score=True)
    grid_search_elastic_net.fit(X,salesPrice)
    grid_search_elastic_net.best_params_

In [11]:
finalAlpha = [1] + list(np.linspace(5,50,10))

In [12]:
grid_para_elsatic_net_final = {'alpha': finalAlpha, 'l1_ratio': [0.0]}

In [13]:
grid_search_elastic_net = ms.GridSearchCV(elasticNet, grid_para_elsatic_net_final, scoring='accuracy', cv=5, n_jobs=-1, return_train_score=True)
grid_search_elastic_net.fit(X,salesPrice)
grid_search_elastic_net.best_params_

/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the train scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 797266995580.6594, tolerance: 1180153387.246548
  model = cd_fast.enet_coordinate_descent(


{'alpha': 1, 'l1_ratio': 0.0}

In [14]:
if exists('../data/sigCorrDictionary.json'):
    with open('../data/sigCorrDictionary.json') as d:
        sigCorrDictionary = json.load(d)
else:
    correlationList = []
    pairList = []
    pValList = []
    for i in range(len(X.columns)):
        for j in range(len(X.columns)):
            if i<j:
                iColumn = X.columns[i]
                jColumn = X.columns[j]
                pairList.append(iColumn+'-'+jColumn)
                corrVal,pVal = stats.spearmanr(X[iColumn],
                                              X[jColumn])
                correlationList.append(corrVal)
                pValList.append(pVal)

    amesHousingCorreltaion = pd.DataFrame({"pair":pairList,
                                    "corr":correlationList,
                                    "pVal":pValList
                                   },columns=["pair","corr","pVal"]).sort_values(by=["pVal","corr"],
                                                                                 ascending=[True,False])

    amesHousingSigCorrs = amesHousingCorreltaion[(np.isnan(amesHousingCorreltaion['pVal'])==False) & (amesHousingCorreltaion['pVal'] < 0.05)]
    absRValues = abs(amesHousingSigCorrs['corr'])
    amesHousingSigCorrs = amesHousingSigCorrs[abs(amesHousingSigCorrs['corr']) > absRValues.quantile(.975)]

    sigCorrDictionary = {}
    for col in X.columns:
        correlatedRelations = (amesHousingSigCorrs[amesHousingSigCorrs['pair'].str.contains(col)])["pair"]
        correlatedColumns = [name.replace("-","").replace(col,"") for name in correlatedRelations]
        relatedDummies = []
        for corrCol in correlatedColumns:
            if corrCol in relatedDummiesDictionary.keys():
                relatedDummies = relatedDummies + relatedDummiesDictionary[corrCol]
        correlatedColumns = list(set(correlatedColumns + relatedDummies))
        sigCorrDictionary[col] = correlatedColumns
    with open('../data/sigCorrDictionary.json', 'w') as fp:
        json.dump(sigCorrDictionary, fp)

In [15]:
baseV1 = pd.DataFrame()
prebaseV2 = pd.DataFrame()
baseV2 = pd.DataFrame()

linmodel = lm.LinearRegression()
linmodel.fit(np.log(X[['GrLivArea']]),np.log(salesPrice))
baseV1[f'GrLivArea^{round(linmodel.coef_[0],2)}'] = X['GrLivArea'] ** round(linmodel.coef_[0],2)

linmodel = lm.LinearRegression()
XWithNeighborhood = pd.merge(X,housingTrainWithDummies[["PID","Neighborhood"]],how='left', left_on='PID', right_on='PID')
Nbr_quantile = XWithNeighborhood.groupby('Neighborhood')['GrLivArea'].quantile(0.8)
joint_Nbr_quantile = pd.merge(XWithNeighborhood[['Neighborhood']], Nbr_quantile, how='left', left_on='Neighborhood', right_index=True)
smaller_home = XWithNeighborhood.GrLivArea < joint_Nbr_quantile.GrLivArea

small = []
large = []
for i,isSmall in enumerate(smaller_home):
    if isSmall:
        small.append(XWithNeighborhood.GrLivArea[i])
        large.append(0)
    else:
        small.append(0)
        large.append(XWithNeighborhood.GrLivArea[i])
linmodel.fit(np.log(XWithNeighborhood[["GrLivArea"]])[smaller_home],np.log(salesPrice)[smaller_home.values])
baseV2[f'smallGrLivArea^{round(linmodel.coef_[0],2)}'] = small ** round(linmodel.coef_[0],2)
linmodel.fit(np.log(XWithNeighborhood[["GrLivArea"]])[~smaller_home],np.log(salesPrice)[~smaller_home.values])
baseV2[f'largeGrLivArea^{round(linmodel.coef_[0],2)}'] = large ** round(linmodel.coef_[0],2)

In [16]:
elasticNet = ElasticNet(alpha=1000,l1_ratio=0.0)
results = cross_validate(elasticNet,baseV1,salesPrice)
np.mean(results['test_score'])

/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2258475231667.5415, tolerance: 933121559.0304306
  model = cd_fast.enet_coordinate_descent(
/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2213165450121.7876, tolerance: 933224897.8421112
  model = cd_fast.enet_coordinate_descent(
/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2374646029616.0728, tolerance: 966845617.2990716
  model = cd_fast.enet_coordinate_descent(
/Users/garysimmons/opt/anaconda3/lib/python3.8/site-

0.5325034775137362

In [17]:
elasticNet = ElasticNet(alpha=1000,l1_ratio=0.5)
results = cross_validate(elasticNet,baseV2,salesPrice)
np.mean(results['test_score'])

0.5809009293889638

In [18]:
availabilityList = list(X.columns)

In [19]:
availabilityList = util.removeDummiesAndCorrelatedFeaturesFromAvailabilityList(availabilityList,"GrLivArea")

In [20]:
f_value,p_value = f_regression(X,salesPrice)
fValuesSorted = pd.DataFrame({"colName":list(X.columns),
                             "f_value":f_value},
                            columns=["colName","f_value"]).sort_values(by=["f_value"],
                                                                                 ascending=[False])

/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:301: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms


In [21]:
xTrain = baseV2
elasticNet = ElasticNet(alpha=1000,l1_ratio=0.5)
results = cross_validate(elasticNet,xTrain,salesPrice,return_train_score=True)
trainScore = np.mean(results['train_score'])
testScore = np.mean(results['test_score'])
bestScore = testScore
modelEvolution = pd.DataFrame({"features":["\n".join(list(xTrain.columns))],
                              "train_score":[trainScore],
                              "test_score":[testScore],
                              "test_score_lift":[0.0]},
                             columns = ["features","train_score","test_score","test_score_lift"])
util.pretty_print(modelEvolution)

,features,train_score,test_score,test_score_lift
0,smallGrLivArea^1.0largeGrLivArea^1.24,0.583105,0.580901,0.0


In [22]:
!rm -r ../images/normalCheck
!rm -r ../images/isHeteroskedastic
!rm -r ../images/isHomoSkedastic
!rm -r ../images/notCentered

!mkdir ../images/normalCheck
!mkdir ../images/isHeteroskedastic
!mkdir ../images/isHomoSkedastic
!mkdir ../images/notCentered

In [23]:
diverseNumbers = [col for col in fValuesSorted.colName if (col not in relatedDummiesDictionary.keys() and len(list(set(X[col].values))) > 15)]
nonDiverseNumbers = [col for col in fValuesSorted.colName if (col not in relatedDummiesDictionary.keys() and len(list(set(X[col].values))) <= 15)]
dummyColumns = [col for col in fValuesSorted.colName if col in relatedDummiesDictionary.keys()]
prioritizedList = diverseNumbers + nonDiverseNumbers + dummyColumns

In [24]:
for i,col in enumerate(prioritizedList):
    if col in availabilityList:
        if col in relatedDummiesDictionary.keys():
            print(f'{i*100.0/len(fValuesSorted.colName)}% complete. Sorting {col}')
            newX = pd.concat([xTrain,X[relatedDummiesDictionary[col]]],axis=1)
        else:
            print(f'{i*100.0/len(fValuesSorted.colName)}% complete. Computing {col}')
            valueSet = list(set(X[col].values))
            if (len(valueSet) <= 15):
                newX = util.engineerSmallFeature(xTrain,X[col],salesPrice,str(col))
            elif util.IsHomoskedastic(X[col],salesPrice,str(col)):
                newX = xTrain.copy()
                newX[col] = X[col]
            else:
                newX = util.engineerFeature(xTrain,X[col],salesPrice,str(col))
        if (type(newX) != type(None)):
            elasticNet = ElasticNet(alpha=1000,l1_ratio=0.5)
            results = cross_validate(elasticNet,newX,salesPrice,return_train_score=True)
            trainScore = np.mean(results['train_score'])
            testScore = np.mean(results['test_score'])
            if testScore > bestScore:
                scoreLift = testScore - bestScore
                bestScore = testScore
                xTrain = newX
                newModel = pd.DataFrame({"features":["\n".join(list(xTrain.columns))],
                              "train_score":[trainScore],
                              "test_score":[testScore],
                              "test_score_lift":[scoreLift]},
                             columns = ["features","train_score","test_score","test_score_lift"])
                modelEvolution = pd.concat([modelEvolution,newModel],axis=0)
                availabilityList = util.removeDummiesAndCorrelatedFeaturesFromAvailabilityList(availabilityList,col)
print('100% complete!')
modelEvolution.reset_index()
util.pretty_print(newModel)

0.3472222222222222% complete. Computing TotalBsmtSF
1.3888888888888888% complete. Computing YearBuilt
1.7361111111111112% complete. Computing MasVnrArea
2.0833333333333335% complete. Computing YearRemodAdd
2.4305555555555554% complete. Computing GarageYrBlt
2.7777777777777777% complete. Computing BsmtFinSF1
3.125% complete. Computing WoodDeckSF
3.4722222222222223% complete. Computing OpenPorchSF
4.166666666666667% complete. Computing LotArea
4.513888888888889% complete. Computing PID
4.861111111111111% complete. Computing LotFrontage
5.208333333333333% complete. Computing BsmtUnfSF
5.555555555555555% complete. Computing ScreenPorch
5.902777777777778% complete. Computing EnclosedPorch
6.25% complete. Computing Unnamed: 0.1
6.597222222222222% complete. Computing ISUDistance
6.944444444444445% complete. Computing Unnamed: 0
7.291666666666667% complete. Computing MiscVal
7.638888888888889% complete. Computing 3SsnPorch
7.986111111111111% complete. Computing LowQualFinSF
8.333333333333334% 

../util/util.py:155: RuntimeWarning: divide by zero encountered in power
  xPow = newX ** power
../util/util.py:155: RuntimeWarning: divide by zero encountered in power
  xPow = newX ** power


12.847222222222221% complete. Computing PoolArea
13.194444444444445% complete. Computing MoSold
13.541666666666666% complete. Computing YrSold
13.88888888888889% complete. Sorting ExterQual_TA
16.666666666666668% complete. Sorting Neighborhood_NridgHt
17.01388888888889% complete. Sorting MasVnrType_None
17.36111111111111% complete. Sorting BsmtExposure_Gd
17.708333333333332% complete. Sorting GarageType_Detchd
20.833333333333332% complete. Sorting HeatingQC_TA
21.180555555555557% complete. Sorting LotShape_Reg
21.52777777777778% complete. Sorting MSZoning_RM
22.569444444444443% complete. Sorting PavedDrive_Y
22.916666666666668% complete. Sorting GarageCond_TA
23.61111111111111% complete. Sorting RoofStyle_Hip
24.305555555555557% complete. Sorting CentralAir_Y
24.65277777777778% complete. Sorting SaleType_New
26.73611111111111% complete. Sorting Electrical_SBrkr
30.90277777777778% complete. Sorting Fence_nan
32.291666666666664% complete. Sorting LandContour_HLS
33.333333333333336% compl

,features,train_score,test_score,test_score_lift
0,smallGrLivArea^1.0largeGrLivArea^1.24BsmtUnfSFScreenPorchEnclosedPorchISUDistanceLowQualFinSFBsmtFinSF2BsmtFullBathOverallCondKitchenAbvGr_invbc_l-0.03_m-0.16_b10.25BsmtHalfBath^0.19PoolArea_invbc_l-0.03_m0.0_b10.09MoSold_invbc_l-0.03_m0.0_b10.07YrSoldExterQual_FaExterQual_GdExterQual_TANeighborhood_BluesteNeighborhood_BrDaleNeighborhood_BrkSideNeighborhood_ClearCrNeighborhood_CollgCrNeighborhood_CrawforNeighborhood_EdwardsNeighborhood_GilbertNeighborhood_GreensNeighborhood_GrnHillNeighborhood_IDOTRRNeighborhood_LandmrkNeighborhood_MeadowVNeighborhood_MitchelNeighborhood_NAmesNeighborhood_NPkVillNeighborhood_NWAmesNeighborhood_NoRidgeNeighborhood_NridgHtNeighborhood_OldTownNeighborhood_SWISUNeighborhood_SawyerNeighborhood_SawyerWNeighborhood_SomerstNeighborhood_StoneBrNeighborhood_TimberNeighborhood_VeenkerMasVnrType_BrkFaceMasVnrType_NoneMasVnrType_StoneMasVnrType_nanBsmtExposure_GdBsmtExposure_MnBsmtExposure_NoBsmtExposure_nanGarageType_AttchdGarageType_BasmentGarageType_BuiltInGarageType_CarPortGarageType_DetchdGarageType_nanHeatingQC_FaHeatingQC_GdHeatingQC_PoHeatingQC_TALotShape_IR2LotShape_IR3LotShape_RegMSZoning_C (all)MSZoning_FVMSZoning_I (all)MSZoning_RHMSZoning_RLMSZoning_RMPavedDrive_PPavedDrive_YGarageCond_FaGarageCond_GdGarageCond_PoGarageCond_TAGarageCond_nanRoofStyle_GableRoofStyle_GambrelRoofStyle_HipRoofStyle_MansardRoofStyle_ShedCentralAir_YSaleType_CWDSaleType_ConSaleType_ConLDSaleType_ConLISaleType_ConLwSaleType_NewSaleType_OthSaleType_VWDSaleType_WD Electrical_FuseFElectrical_FusePElectrical_SBrkrElectrical_nanFence_GdWoFence_MnPrvFence_MnWwFence_nanLandContour_HLSLandContour_LowLandContour_LvlBsmtCond_FaBsmtCond_GdBsmtCond_PoBsmtCond_TABsmtCond_nanLotConfig_CulDSacLotConfig_FR2LotConfig_FR3LotConfig_InsideExterCond_FaExterCond_GdExterCond_PoExterCond_TACondition1_FeedrCondition1_NormCondition1_PosACondition1_PosNCondition1_RRAeCondition1_RRAnCondition1_RRNeCondition1_RRNnFunctional_Maj2Functional_Min1Functional_Min2Functional_ModFunctional_SalFunctional_TypAlley_PaveAlley_nanMiscFeature_OthrMiscFeature_ShedMiscFeature_TenCMiscFeature_nan,0.638119,0.630646,1.568221e-09


In [25]:
modelEvolution

,features,train_score,test_score,test_score_lift
0,smallGrLivArea^1.0\nlargeGrLivArea^1.24,0.583105,0.580901,0.000000e+00
0,smallGrLivArea^1.0\nlargeGrLivArea^1.24\nBsmtU...,0.584311,0.581405,5.037254e-04
0,smallGrLivArea^1.0\nlargeGrLivArea^1.24\nBsmtU...,0.586130,0.582765,1.360132e-03
0,smallGrLivArea^1.0\nlargeGrLivArea^1.24\nBsmtU...,0.601704,0.596666,1.390130e-02
0,smallGrLivArea^1.0\nlargeGrLivArea^1.24\nBsmtU...,0.601705,0.596667,7.921900e-07
0,smallGrLivArea^1.0\nlargeGrLivArea^1.24\nBsmtU...,0.609615,0.603503,6.835953e-03
0,smallGrLivArea^1.0\nlargeGrLivArea^1.24\nBsmtU...,0.610525,0.604000,4.973558e-04
0,smallGrLivArea^1.0\nlargeGrLivArea^1.24\nBsmtU...,0.610571,0.604046,4.613115e-05
0,smallGrLivArea^1.0\nlargeGrLivArea^1.24\nBsmtU...,0.610572,0.604047,4.840611e-07
0,smallGrLivArea^1.0\nlargeGrLivArea^1.24\nBsmtU...,0.637872,0.630399,2.635245e-02


## Random Forest

# Testing

## Linear Regression

## Random Forest